### 4. Working with Key/Value Pairs

We will look at a special type of RDD called Key Value Pair RDD which exposes some new set of operations and is most commonly used type of RDD when we need to perform some analytics and aggegation on large set of data. We will see how to partition the data based on key to ensure commonly accesses data resides together on the same partition and there are no shuffles involved in moving large amount of data across the machines over the network. 

These types of RDDs have a special name called ``PairRDD`` and has special features that let us perform aggregation by the key and also join two RDD based on the key. Let us read the a text file and create a ``PairRDD`` using the lower cased first word of each line in the text file and then perform some aggregation operations on them

In [1]:
val src = scala.io.Source.fromURL("https://raw.githubusercontent.com/apache/spark/master/README.md")
val contentRDD = sc.parallelize(src.mkString.split("\n"))
val pairRDD = contentRDD.map(x => (x.split(" ")(0).toLowerCase, 1))
println("Top 10 Words aggregated with their descending value of counts are " + 
pairRDD.reduceByKey((x, y) => x + y).takeOrdered(10)(Ordering.by(x => -x._2)).toList)


Top 10 Words aggregated with their descending value of counts are List((,47), (##,9), (spark,4), (please,4), (you,3), (for,3), (and,2), (to,2), (can,2), (hadoop,,1))



The ``reduceByKey`` is similar to ``reduce`` action we saw previously, except that the reduce is called on values of same key element and **``reduceByKey`` is a transformation**. Note that similar to ``reduce`` the return type of the ``reduceByKey`` is same as the type of the ***value*** in the key value pair. No where in the ``reduceByKey`` we know whats the key associated with the values being reduced. Spark automatically gives an RDD with the key and the reduced values of the key. This special type of reduce function is available only in key value type of RDD.

Since we resulting RDD is a tuple and while we take the top 10 values we want to order by the value, we used ``Ordering.by(x => -x._2)`` for ordering to sort numbers by the second element (value) of the tuple rather than the key (the first word of the line). Since ``Ordering.by`` will sort the numbers by ascending order and we wish to sort the numbers by descending value of the count, we simply flip the sign of the value.

Let us create a small RDD and studying some simple transformation we get in ``PairRDD``

In [2]:
val pairSample = sc.parallelize(List((1, 2), (3, 4), (3, 6)))
println("1. Reducing by key for adding the values for similar key: " + pairSample.reduceByKey(_ + _).collect.toList)
println("2. Group by key:  " + pairSample.groupByKey().collect.toList)
println("3. mapValues by key:  " + pairSample.mapValues(_ + 10).collect.toList)
println("4. flatMapValue by key:  " + pairSample.flatMapValues(x => x to (x + 2)).collect.toList)
println("5. Keys of pair RDD:  " + pairSample.keys.collect.toList)
println("6. Values of pair RDD:  " + pairSample.values.collect.toList)
println("7. sortByKey of pair RDD:  " + pairSample.sortByKey().collect.toList)
println("8. sortByKey of pair RDD in descending order:  " + pairSample.sortByKey(ascending = false).collect.toList)

1. Reducing by key for adding the values for similar key: List((1,2), (3,10))
2. Group by key:  List((1,CompactBuffer(2)), (3,CompactBuffer(4, 6)))
3. mapValues by key:  List((1,12), (3,14), (3,16))
4. flatMapValue by key:  List((1,2), (1,3), (1,4), (3,4), (3,5), (3,6), (3,6), (3,7), (3,8))
5. Keys of pair RDD:  List(1, 3, 3)
6. Values of pair RDD:  List(2, 4, 6)
7. sortByKey of pair RDD:  List((1,2), (3,4), (3,6))
8. sortByKey of pair RDD in descending order:  List((3,4), (3,6), (1,2))



- **reduceByKey**: We have already seen this, after reduceByKey, the RDD is guaranteed not to have tuples with duplicate keys. The values are reduced by the function passed to ``reduceByKeys`` in this case additition.
- **groupByKey**: Similar to ``reduceByKey`` after invoking this method, the tuples in the RDD are guaranteed not to have duplicate keys. The value assiciated with the key is a buffer (a sequence of values) holding all the values assiciated with the key. Thus ``RDD[K, V].groupByKey() -> RDD[K, Seq[V]]``
- **mapValues**: applies the function to the values of in the key value pair essentially tranforming ``(k, v) -> (k, f(v))``
- **flatMapValue**: applies a function ``f(V) => Traversable[U]`` on the values of pair ``RDD[(K, V)]`` to emit the pair ``(K, U)`` for each value in ``Traversable[U]`` 
- **keys**: Simply returns the RDD of keys. ``RDD[(K, V)] => RDD[K]``
- **values**: Simply returns the RDD of values. ``RDD[(K, V)] => RDD[V]``
- **sortByKeys**: Creates an RDD which sorts the RDD by the keys. The parameter ``ascending`` which defaults to true can be used to reverse the order of the sorting

---

We will now look at some operations on pair RDDs.

In [3]:
val anotherPairRDD = sc.parallelize(List((3, 9), (4, 10)))
println("1. Join two RDDs" + pairSample.join(anotherPairRDD).collect.toList)
println("2. Left join two RDDs" + pairSample.leftOuterJoin(anotherPairRDD).collect.toList)
println("3. Left join two RDDs" + pairSample.rightOuterJoin(anotherPairRDD).collect.toList)
println("4. Cogroup join two RDDs" + pairSample.cogroup(anotherPairRDD).collect.toList)

1. Join two RDDsList((3,(4,9)), (3,(6,9)))
2. Left join two RDDsList((1,(2,None)), (3,(4,Some(9))), (3,(6,Some(9))))
3. Left join two RDDsList((4,(None,10)), (3,(Some(4),9)), (3,(Some(6),9)))
4. Cogroup join two RDDsList((4,(CompactBuffer(),CompactBuffer(10))), (1,(CompactBuffer(2),CompactBuffer())), (3,(CompactBuffer(4, 6),CompactBuffer(9))))



- **join**: This operation reduces the result to only those elements whose keys are present in both the RDDs. Thus ``RDD[K, V1] join RDD [K, V2] -> RDD[K, (V1, V2)]``. If we have multiple values of K in either RDDs, we will get all possible combinations of V1 and V2 for the same keys.
- **leftOuterJoin**: Retains all keys in the left RDD. Thus ``RDD[K, V1] leftOuterJoin RDD [K, V2] -> RDD[K, (V1, V2)] -> RDD[K, (V1, Option[V2])]``. The ``Option`` will be ``Some[V2]`` if the matching key is present in right RDD else ``None``.
- **rightOuterJoin**: Similar to leftOuterJoin, except that the value tuple is ``(Option[V1], V2)`` instead of ``(V1, Option[V2])``
- **cogroup**: This is similar to ``groupByKey`` operation on a single RDD except that the result RDD is  ``RDD[K, (Seq[V1], Seq[V2])]``. If the key is not present in either RDD, the corresponding Seq will be empty.

Following example shows how we can use combination of multiple transformation to compute the mean per key. The following code is pretty efficient even onlarge datasets if we have the RDD partitioned across the cluster by keys in which case only the final results are transported to the driver code.

In [4]:
val pairSample1 = sc.parallelize(List((1, 2), (3, 4), (3, 6), (4, 2), (3, 1)))
pairSample1.mapValues((_, 1)).reduceByKey{
    case ((value1, count1), (value2, count2)) => (value1 + value2, count1 + count2)
}.mapValues{case (sum, count) => sum / count.toFloat}.collect.toList

List((4,2.0), (1,2.0), (3,3.6666667))

Following example will perform the same operation but using ``combineByKey`` operation which lets us transformation return different type than the input RDD.

In [5]:
pairSample1.combineByKey(
    (_, 1), //Create Combiner
    (acc:(Int,Int), v:Int) => (acc._1 + v, acc._2 + 1), //Merge Values with combiner
    (acc1:(Int, Int), acc2:(Int, Int)) => (acc1._1 + acc2._1, acc1._2 + acc2._2) //Merge combiners
).mapValues{case (sum, count) => sum / count.toFloat}.collect.toList

List((4,2.0), (1,2.0), (3,3.6666667))


The ``combineByKey`` is equivalent of the ``mapValues`` followed by ``reduceByKey``. As we see in the above example, the results are identical in both invocations. The ``combineByKey`` method takes three parameters. 
- The first one, creates a combiner, which has the same return type as the return type of the RDD. For mean value, we simply emit the value and the current count for the given key, initialized to 1
- The merge function which merges the value of the RDD to the accumulated combined values. In our case the RDD has Int values and combiner creates has (Int, Int) type. We simply add the current value to the first element, which is the running total and increment the second value of the tuple, the running count by 1.
- The third function is needed when we have the same key distributed across the cluster. Each cluster will give the total of the values and the number of occurance for that key on them We are interested in the aggregated value across cluster and thus this function simply merges the combiners. In our case we just add the corresponding values and then running totals.

We will look at few actions available with us for Pair RDDs. The results are self explanatory.

In [6]:
println("1. countByKey gives " + pairSample.countByKey())
println("2. collectAsMap gives " + pairSample.collectAsMap())
println("3. lookup(1) gives " + pairSample.lookup(1))
println("4. lookup(3) gives " + pairSample.lookup(3))

1. countByKey gives Map(1 -> 1, 3 -> 2)
2. collectAsMap gives Map(1 -> 2, 3 -> 6)
3. lookup(1) gives WrappedArray(2)
4. lookup(3) gives WrappedArray(4, 6)


---

#### Data Partitioning

In distributed applications data is placed across the cluster and chosing how the data is partitioned is important to minimize moving large amount of data over the network. For RDDs which are used once, partitioning doesnt make sense but for RDDs which are used over and over again to be joined with other RDDs and cached, partitioning the data once makes a big difference.

Spark allows us to partition pair RDDs by their key. The data can be partitioned either by the range partitioner or by hash partitioner. For a cluster with 100 partitions, all data with same ``hash(key) % 100`` will end up on same partition.

Lets look at this hypothetical code sample

```

val userData = sc.sequenceFile[UserId, UserInfo]("hdfs://...").persist()

def processNewLogs(logFileName: String) {
   val events = sc.sequenceFile[UserId, LinkInfo]("hdfs://...")
   val joined = userData.join(events)  //RDD[UserId, (UserInfo, LinkInfo)]
   val offTopicVisits = joined.filter {
     case (userId, (userInfo, linkInfo)) => !userInfo.topics.contains(linkInfo.topics)
   }.count()
   println("Visits to non subscribed topics are " + offTopicVisits)
}

```

Though the above code works fine, it isnt efficient. The ``userData`` is frequently used to be joined with the events periodically. Since the data is not partitioned when we join the ``userData`` and ``events`` there would be a lot of shuffles slowing down the join operation. An efficient way would be to partition and persist the ``userData`` so that the ``userData`` is sticky and doesnt move around but the events data is moved to to the same partition as the ``userData`` minimizing the shuffles. Partitioning the events data doesnt make sense as it is used only once and is local to ``processNewLogs``.

Following change to the way ``userData`` RDD is created will fix the problem

``val userData = sc.sequenceFile[UserId, UserInfo]("hdfs://...").partitionBy(new HashPartitioner(100)).persist()``

Note that we persist the RDD after the partition call as partition is a transformation.

The number 100 to the ``HashPartitioner`` is the number of partitions and should be around the number of cores in the cluster.

By default the RDD is not partitioned and some operations like ``sortByKey`` and ``groupByKey`` implicitly partition the data. We will demo this by the following code snippet

In [7]:
import org.apache.spark.HashPartitioner
println("1. Partitioner by default is " + pairSample.partitioner)
println("2. Partition by explicit partioner gives " + pairSample.partitionBy(new HashPartitioner(4)).partitioner)
println("3. Partitioner after groupByKey is " + pairSample.groupByKey.partitioner)
println("4. Partitioner after sortByKey is " + pairSample.sortByKey().partitioner)
println("5. Partitioner on map after groupByKey is " + pairSample.groupByKey.map(x => x).partitioner)
println("6. Partitioner on mapValues after groupByKey is " + pairSample.groupByKey.mapValues(x => x).partitioner)
println("7. Partitioner on mapValues after sortByKey is " + pairSample.sortByKey().mapValues(x => x).partitioner)

1. Partitioner by default is None
2. Partition by explicit partioner gives Some(org.apache.spark.HashPartitioner@4)
3. Partitioner after groupByKey is Some(org.apache.spark.HashPartitioner@2)
4. Partitioner after sortByKey is Some(org.apache.spark.RangePartitioner@8ed)
5. Partitioner on map after groupByKey is None
6. Partitioner on mapValues after groupByKey is Some(org.apache.spark.HashPartitioner@2)
7. Partitioner on mapValues after sortByKey is Some(org.apache.spark.RangePartitioner@8ed)



- What we see above is that by default the partioner for the RDD is None.
- On pair RDDs we have a method partitionBy to let us partition the RDD explicitly.
- Operations like ``groupByKey`` partitions the resulting RDD. We can pass either the number or partitions or a ``Partitioner`` explicitly to ``groupByKey`` if we dont want to use the default.
- The ``sortByKey`` operation similarly partition but with ``RangePartitioner``.
- Operations on RDD (not the ones on pair RDD) like ``map`` dont retain the partitions as the map operation possibly change the key.
- We we just intend to map the values, using operations like ``mapValues`` is preferred as that retains the partioner from the previous transformation.

---

If the RDD is partitioned then running operations that execute on on RDD like ``groupByKey``, ``reduceByKey``, ``lookup`` etc will execute the operations locally with no data sent out to network. If we have join operations and operations like ``cogroup``,  pre-partitioning will not cause contents of one RDD to be shuffled and if both RDDs are partitioned and present on the same machine, then no data will be sent over network.

Operations like ``cogroup``, ``groupWith``, ``join``, ``leftOuterJoin``, ``rightOuterJoin``, ``groupByKey``, ``reduceByKey``, ``combineByKey``, ``partitionBy``, ``sort`` set the partioner on the RDD where and operations like ``mapValues``, ``flatMapValues`` retain the partitioners. 

---

Lets implement page rank algorithm on a small graph below



In [8]:
import org.apache.spark.rdd.RDD

def pageRank(links: RDD[(Char, Seq[Char])], ranks: RDD[(Char, Double)], numIters:Int = 100, f: Double => Double = x => x)= {
    var rank = ranks
    for (i <- 0 to numIters) {
        var contributions = links.join(rank).flatMap {
            case (page, (links, rank)) => links.map(dest => (dest, rank / links.size))
        }
        rank = contributions.reduceByKey(_ + _).mapValues(v => f(v))
    }
    rank.collect.toList
}

In [9]:
val links = sc.parallelize(List(('A', Seq('B', 'C')), 
                        ('B', Seq('D')), 
                        ('C', Seq('A', 'B', 'D')), 
                        ('D', Seq('C')))).partitionBy(new HashPartitioner(4))
var ranks = links.mapValues(_ => 0.25)
//var ranks = links.mapValues(_ => 1.0)

pageRank(links, ranks, 1)

List((D,0.3333333333333333), (A,0.125), (B,0.16666666666666666), (C,0.375))


The above implementation is giving us the correct results as per the algorithm explained at this. 

[![Page Rank](https://img.youtube.com/vi/P8Kt6Abq_rM/0.jpg)](https://www.youtube.com/watch?v=P8Kt6Abq_rM)

We will next use a modest sized dataset and validate our results.

TODO: Implement PageRank with a real dataset

---

### Using custom partitioners.

We can develop our own partitioners as follows

```
 class DomainNamePartitioner(numParts:Int) extends Partitioner {
    override def numPartitions = numParts
    override def getPartition(key: Any): Int = {
        val domain = new java.net.URL(key.toString).getHost()
        val code = domain.hashCode % numPartitions
        if (code < 0) {
           code + numPartitions 
        } else {
           code
        }
    }
    
    override def equal(other: Any): Boolean = other match {
       case dnp: DomainNamePartitioner => dnp.numPartitions == numPartitions
       case _ => false
    }
 }
```